In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

data = xr.open_dataset("../1_data_processing/1_1_Forcing/all_stations_cmems.zarr", engine="zarr")
data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset("../1_data_processing/1_1_Forcing/all_stations_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time", how="all")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
_ = cafe_npp.load()

In [3]:
data

<xarray.Dataset> Size: 13MB
Dimensions:              (time: 8767, depth: 3, latitude: 3, longitude: 3)
Coordinates:
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 24B 22.75 31.6 50.0
  * longitude            (longitude) float64 24B -158.0 -145.0 -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (time, depth, latitude, longitude) float64 2MB 23.54...
    U                    (time, depth, latitude, longitude) float64 2MB 0.015...
    V                    (time, depth, latitude, longitude) float64 2MB -0.07...
    mnkc_epi             (time, latitude, longitude) float64 631kB 0.2627 ......
    mnkc_hmlmeso         (time, latitude, longitude) float64 631kB 0.7987 ......
    mnkc_lmeso           (time, latitude, longitude) float64 631kB 2.157 ... nan
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 631kB 0.4632 ......
    mnkc_umeso           (time, latitude, longitude) float64 631kB 0.4843 ......
    npp                  (time, latitude, longitude) float64 631kB 316.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 2MB 117.3...
    zeu                  (time, latitude, longitude) float64 631kB 78.17 ... nan
    zooc                 (time, latitude, longitude) float64 631kB 0.2832 ......
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

In [4]:
cafe_npp

<xarray.Dataset> Size: 701kB
Dimensions:    (time: 8767, latitude: 3, longitude: 3)
Coordinates:
  * latitude   (latitude) float64 24B 22.75 31.6 50.0
  * longitude  (longitude) float64 24B -158.0 -150.0 -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 631kB 306.4 nan ... nan nan
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [5]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../1_data_processing/1_1_Forcing/Bats_obs.zarr", engine="zarr")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)

obs_hot = xr.open_dataset("../1_data_processing/1_1_Forcing/Hot_obs.zarr", engine="zarr")
obs_hot = update_layer(obs_hot, epipelagic_size)
obs_hot = Observation(obs_hot)

obs_papa = xr.open_dataset("../1_data_processing/1_1_Forcing/Papa_obs.zarr", engine="zarr")
obs_papa = update_layer(obs_papa, epipelagic_size)
obs_papa = Observation(obs_papa)

observations = [obs_bats, obs_hot, obs_papa]

Create structure for SeapoPym simulation.


In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=data["T"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12, timestep=1),
)

2024-11-06 11:57:27,885 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [7]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [8]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [9]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=1,
    INDPB=0.5,
    CXPB=0.5,
    MUTPB=0.5,
    NGEN=10,
    POP_SIZE=200,
    cost_function_weight=(-(1 / 3), -(1 / 3), -(1 / 3)),
    hall_of_fame_size=2000,
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [10]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [ ]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57883 instead
  warnings.warn(


2024-11-06 16:06:43,752 - distributed.scheduler - WARNING - Worker failed to heartbeat for 423s; attempting restart: <WorkerState 'tcp://127.0.0.1:57895', name: 3, status: running, memory: 0, processing: 0>
2024-11-06 16:06:43,753 - distributed.scheduler - WARNING - Worker failed to heartbeat for 423s; attempting restart: <WorkerState 'tcp://127.0.0.1:57896', name: 2, status: running, memory: 0, processing: 0>
2024-11-06 16:06:43,754 - distributed.scheduler - WARNING - Worker failed to heartbeat for 423s; attempting restart: <WorkerState 'tcp://127.0.0.1:57897', name: 0, status: running, memory: 0, processing: 0>
2024-11-06 16:06:43,757 - distributed.scheduler - WARNING - Worker failed to heartbeat for 423s; attempting restart: <WorkerState 'tcp://127.0.0.1:57898', name: 1, status: running, memory: 0, processing: 0>
2024-11-06 16:06:45,536 - distributed.nanny - WARNING - Restarting worker
2024-11-06 16:06:45,609 - distributed.nanny - WARNING - Restarting worker
2024-11-06 16:06:45,627 

And watch the magic on the Dask dashboard :


In [12]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57883/status,
Dashboard: http://127.0.0.1:57883/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57884,Workers: 4
Dashboard: http://127.0.0.1:57883/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57897,Total threads: 2
Dashboard: http://127.0.0.1:57899/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:57887,


And execute the process.


In [13]:
viewer = genetic_algo.optimize()

2024-11-06 11:57:29,602 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:57:29,603 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:57:29,603 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:57:29,604 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:57:29,606 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:57:29,606 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:57:29,608 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:57:29,611 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:57:29,615 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-06 11:57:29,620 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-06 11:57:29,625 :: Seapodym ::  DEBU

In [14]:
viewer.logbook

,gen,nevals,avg,std,min,max,nvalide,ninvalide
0,0,200,0.000016,0.000031,1.299342e-07,0.000458,396,204
1,1,165,0.000015,0.000020,1.320544e-07,0.000096,504,96
2,2,154,0.000015,0.000020,1.253182e-07,0.000109,531,69
3,3,153,0.000016,0.000023,1.265361e-07,0.000260,555,45
4,4,157,0.000015,0.000022,1.232191e-07,0.000237,525,75
5,5,162,0.000015,0.000019,1.232191e-07,0.000072,555,45
6,6,152,0.000016,0.000026,1.192298e-07,0.000364,558,42
7,7,145,0.000015,0.000020,1.192298e-07,0.000080,573,27
8,8,152,0.000016,0.000021,1.192298e-07,0.000103,552,48
9,9,148,0.000016,0.000022,1.192298e-07,0.000157,579,21


Finaly here is the result :


In [15]:
viewer.box_plot(4)

In [ ]:
viewer.parallel_coordinates(
    1000,
    color_continuous_scale=[
        [0, "rgba(255,0,0,0)"],
        [0.7, "rgba(255,0,0,0.5)"],
        [1, "rgba(50, 255, 50, 1)"],
    ],
)

In [17]:
viewer.hall_of_fame.sort_values(by="fitness")

,D1N1_tr_max,D1N1_tr_rate,D1N1_inv_lambda_max,D1N1_inv_lambda_rate,D1N1_energy_coefficient,D2N1_tr_max,D2N1_tr_rate,D2N1_inv_lambda_max,D2N1_inv_lambda_rate,D2N1_energy_coefficient,fitness
0,16.798279,-0.639497,72.409356,0.094435,0.122714,1.502236,-0.590203,48.982891,0.064481,0.060071,1.192298e-07
1,16.798279,-0.639497,72.409356,0.094435,0.128090,49.663412,-0.590203,48.982891,0.064481,0.060071,1.193964e-07
2,48.737987,-0.993749,81.811224,0.085792,0.100183,31.636778,-0.252291,41.328609,0.241363,0.785127,1.207091e-07
3,46.808425,-0.946641,65.812772,0.078454,0.100257,4.692802,-0.110341,54.020231,0.104893,0.095249,1.209311e-07
4,40.160946,-0.673049,72.409356,0.112592,0.167384,41.713729,-0.726043,48.982891,0.064481,0.060071,1.209610e-07
...,...,...,...,...,...,...,...,...,...,...,...
1499,15.605896,-0.106797,65.812772,0.003191,0.412656,12.118062,-0.571246,78.031733,0.106011,0.084813,8.548295e-05
1500,43.499156,-0.338335,49.521334,0.007969,0.746347,12.317363,-0.367816,54.493833,0.485336,0.228322,1.335381e-04
1501,48.902984,-0.029942,43.167450,0.828267,0.256303,32.536692,-0.948440,96.161245,0.005885,0.526513,1.340056e-04
1502,12.136194,-0.569212,83.149540,0.004760,0.506416,48.824147,-0.957915,44.472321,0.106011,0.084813,1.946815e-04
